# Vision2Clean AI Notebook

Welcome! This notebook guides you through setting up a scalable, human-friendly real-time waste detection and mapping system using YOLO, OpenCV, and Folium.

## Getting Started: Your Smart Waste Detection System

This section will help you configure and launch your own real-time waste detection pipeline. Just follow the steps and customize as needed!

In [ ]:
# --- User Configuration ---
# Set your model path, video source, and camera GPS location here.
MODEL_PATH = 'yolov8n.pt'  # Path to your YOLO model (change if you have a custom model)
VIDEO_SOURCE = 0  # 0 = webcam, or provide a video file path or IP camera URL
CAMERA_GPS = (12.9716, 77.5946)  # (latitude, longitude) of your camera location

In [ ]:
# --- Model and Video Setup ---
import cv2
from ultralytics import YOLO

try:
    model = YOLO(MODEL_PATH)
    print(f'Model loaded: {MODEL_PATH}')
except Exception as e:
    raise RuntimeError(f'Failed to load YOLO model: {e}')

cap = cv2.VideoCapture(VIDEO_SOURCE)
if not cap.isOpened():
    raise RuntimeError(f'Cannot open video source: {VIDEO_SOURCE}')
print(f'Video source opened: {VIDEO_SOURCE}')

In [ ]:
# --- Real-Time Detection Loop ---
import time
import logging

logging.basicConfig(filename='vision2clean.log', level=logging.INFO, 
                    format='%(asctime)s %(levelname)s:%(message)s')

def process_frame(frame, model, gps, frame_num):
    """Detect objects in a frame, draw boxes, and return detection info."""
    results = model(frame)
    boxes = results[0].boxes
    detections = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        label = model.names[cls] if hasattr(model, 'names') else str(cls)
        # Draw bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        detection = {'frame': frame_num, 'label': label, 'confidence': conf, 'gps': gps}
        detections.append(detection)
        logging.info(f"Frame {frame_num}: {label} ({conf:.2f}) at {gps}")
    return frame, detections

def run_detection_loop(model, cap, gps):
    """Run the main detection loop, display results, and log detections."""
    detections_log = []
    frame_num = 0
    start_time = time.time()
    print('Starting detection. Press "q" to quit.')
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print('No frame received. Exiting...')
                break
            processed_frame, detections = process_frame(frame, model, gps, frame_num)
            detections_log.extend(detections)
            frame_num += 1
            # Show FPS
            elapsed = time.time() - start_time
            fps = frame_num / elapsed if elapsed > 0 else 0
            cv2.putText(processed_frame, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
            cv2.imshow('Vision2Clean Detection', processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print('Detection stopped by user.')
                break
    except Exception as e:
        logging.error(f"Error during detection: {e}")
        print(f'Error: {e}')
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print('Resources released.')
    print(f'Total frames: {frame_num}, Total detections: {len(detections_log)}')
    return detections_log

detections_log = run_detection_loop(model, cap, CAMERA_GPS)

In [ ]:
# --- Save and Visualize Detections ---
# Install pandas and folium if needed: %pip install pandas folium
import pandas as pd
import folium

if detections_log:
    df = pd.DataFrame(detections_log)
    df.to_csv('detections_log.csv', index=False)
    print(f'Detections saved to detections_log.csv ({len(df)} records)')
    # Create map and add markers for each detection
    detection_map = folium.Map(location=CAMERA_GPS, zoom_start=12)
    for _, row in df.iterrows():
        folium.Marker(location=row['gps'], popup=f"{row['label']} ({row['confidence']:.2f})").add_to(detection_map)
    print('Map with detections ready below:')
    detection_map
else:
    print('No detections to save or visualize.')

In [ ]:
# --- Install Required Packages (run if needed) ---
# %pip install ultralytics opencv-python folium pandas

In [ ]:
# --- Check Library Versions ---
from ultralytics import YOLO
import cv2
import folium
import pandas as pd

print('Ultralytics YOLO version:', YOLO.__module__)
print('OpenCV version:', cv2.__version__)
print('Folium version:', folium.__version__)
print('Pandas version:', pd.__version__)

In [ ]:
# Load default YOLOv8n model (change to your custom model path if available)
model = YOLO('yolov8n.pt')
print('Model loaded successfully')

In [ ]:
# Minimal detection example (using the image_path variable)
results = model("D:\Documents\Portfolio.github\Vision2Clean-AI\sample.jpg")
results[0].show()  # Display detection results
print(results[0].boxes)  # Print detected bounding boxes

In [ ]:
# Minimal map visualization example
# Replace with actual coordinates and detection info
m = folium.Map(location=[12.9716, 77.5946], zoom_start=12)
folium.Marker([12.9716, 77.5946], popup='Detected Waste: Plastic').add_to(m)

m